In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
from google.colab import drive
drive.mount('/content/drive') # give access to Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
input_path_original_inflation = '/content/drive/MyDrive/Colab Notebooks/Gradu/Data/Original inflation/'
input_path = '/content/drive/MyDrive/Colab Notebooks/Gradu/Data/2_Stationary/'

In [5]:
countries = ['Finland', 'United States', 'Germany', 'France', 'Spain', 'Italy', 'Netherlands', 'Sweden', 'Belgium', 'Denmark', 'Austria', 'Poland']
transformations = ['FD', 'FD', 'none', 'FD', 'FD', 'FD', 'FD', 'none', 'none', 'FD', 'none', 'FD']

In [6]:
macro_variables = ['inflation', 'unemployment', 'imports', 'exports', 'bond_yield', 'exchange', 'ppi', 'bci', 'cci', 'construction', 'manufacturing', 'share_prices', 'gdp', 'house_prices', 'investment', 'domestic_demand']
common_variables = ['oil', 'silver', 'eurusd', 'eurcfh', 'spx', 'world']
all_variables = macro_variables + common_variables

In [7]:
def get_data(variable, path_in = input_path):
    input_file = path_in + variable +'.csv'
    data = pd.read_csv(input_file, header=0, index_col=0)
    data.index = pd.to_datetime(data.index)
    data = data.astype(float)
    data = data.sort_index()
    return data

In [8]:
def drop_constant_columns(data):
    non_constant_columns = data.columns[data.nunique() > 1]
    return data[non_constant_columns].copy()

In [ ]:
# create the data matrix

init_index = get_data("inflation_Finland", input_path)
df = pd.DataFrame(index = init_index.index)

for variable in macro_variables:
    for country in countries:
        data = get_data(variable + "_" + country)
        new_column_name = variable + "_" + country
        data = data.rename({'Data': new_column_name}, axis=1)
        df[new_column_name] = data

for variable in common_variables:
    data = get_data(variable + "_" + 'Finland')
    new_column_name = variable + "_" + 'common'
    data = data.rename({'Data': new_column_name}, axis=1)
    df[new_column_name] = data

df = drop_constant_columns(df)

# Grid Search

This can be a time consuming process. Hence, the results from the grid search are displayed in the next cell after the below cell.

In [ ]:
split_point = len(df)-12 # use last 12 months as the out of sample test dataset
train = df[:split_point]
test = df[split_point:]

scaler = StandardScaler()
normalized_train = scaler.fit_transform(train)

max_factor = 12
max_order = 15

best_aic = float('inf')
best_params = None

for factor in range(max_factor+1):
    for order in range(max_order+1):
        model_params = (factor, order)
        print("params: " + str(model_params))

        try:
            # train the model
            factor_model = sm.tsa.DynamicFactorMQ(endog=normalized_train, factors=factor, factor_orders=order, standardize=False)
            print("fitting model with params now")
            fit = factor_model.fit(maxiter=500, disp=10)
            print("fitting is done")
            aic = fit.aic
            print("aic: " + str(aic))
            if aic < best_aic:
                best_aic = aic
                best_params = model_params
                print("best AIC: " + str(best_aic))
                print("best params: " + str(best_params))
                print()
            else:
                print()
        except:
            print("there was an exception")
            print()
            continue

print("best model params")
print(best_params)

In [10]:
best_params = (8,12)

# Forecasts for 2022

In [11]:
horizons = [1,2,3,6,9,12]

split_point = len(df)-12 # use last 12 months as the out of sample test dataset
train = df[:split_point]
test = df[split_point:]

scaler = StandardScaler()
normalized_train = scaler.fit_transform(train)

factor = best_params[0]
order = best_params[1]

# train the model
model = sm.tsa.DynamicFactorMQ(endog=normalized_train, factors=factor, factor_orders=order, standardize=False)
fit = model.fit(maxiter=500, disp=10)

EM start iterations, llf=-36223
EM iteration 10, llf=-32331, convergence criterion=5.8406e-05
EM iteration 20, llf=-32323, convergence criterion=1.177e-05
EM iteration 30, llf=-32321, convergence criterion=5.3134e-06
EM iteration 40, llf=-32320, convergence criterion=3.0673e-06
EM iteration 50, llf=-32319, convergence criterion=2.0161e-06
EM iteration 60, llf=-32318, convergence criterion=1.4368e-06
EM iteration 70, llf=-32318, convergence criterion=1.0822e-06
EM converged at iteration 74, llf=-32318, convergence criterion=9.7783e-07 < tolerance=1e-06


In [13]:
for (country, transformation) in zip(countries, transformations):
    warnings.filterwarnings('ignore')

    print("country: " + str(country))
    print()

    split_point = len(df)-12 # use last 12 months as the out of sample test dataset
    train = df[:split_point]
    test = df[split_point:]

    # original inflation for reversing stationarity and calculating the RMSE
    original_inflation = "original_inflation_" + str(country)
    data_original = get_data(original_inflation, input_path_original_inflation)
    test_original = data_original[split_point:] # original inflation for country for 2022

    # horizon is how far we are forecasting (no need to do this in a loop really but for clarity it is done that way)
    for horizon in horizons:
        print("horizon: " + str(horizon))
        horizon_test = test_original[:horizon]

        # out-of-sample forecast
        predictions_normalized = fit.forecast(horizon) # do this outside the loop maybe
        predictions_denormalized = scaler.inverse_transform(predictions_normalized)

        #### this will contain all predictions from which we extract only a single column of country specific inflation predictions
        forecast_all_df = pd.DataFrame(predictions_denormalized, index=horizon_test.index, columns=test.columns+ '_pred')

        ## this will contain only country specific actual and forecast
        forecast_df = pd.DataFrame(index=horizon_test.index)
        forecast_df['Actual'] = horizon_test['Data']
        forecast = forecast_all_df['inflation_' + country + '_pred']

        # we need to reverse the stationarity
        if transformation == "FD":
            last_observed_value = data_original[:split_point]
            last_observed_value = last_observed_value.values
            last_observed_value = last_observed_value[-1] # get the last observed value, i.e. 2021-12
            forecasted_original = []
            for pred in forecast:
                forecasted_value = last_observed_value + pred
                forecasted_original.append(forecasted_value)
                last_observed_value = forecasted_value
            forecast = forecasted_original
            forecast = np.concatenate(forecast)
            forecast_df['Forecast'] = forecast
        else:
            forecast_df['Forecast'] = forecast

        rmse = np.sqrt(mean_squared_error(forecast_df['Actual'], forecast_df['Forecast']))
        print('RMSE: ', round(rmse,3))
        print()
    print()
    print("##################")

country: Finland

horizon: 1
RMSE:  0.753

horizon: 2
RMSE:  0.834

horizon: 3
RMSE:  1.417

horizon: 6
RMSE:  2.378

horizon: 9
RMSE:  2.919

horizon: 12
RMSE:  3.582


##################
country: United States

horizon: 1
RMSE:  0.31

horizon: 2
RMSE:  0.534

horizon: 3
RMSE:  0.903

horizon: 6
RMSE:  1.11

horizon: 9
RMSE:  1.036

horizon: 12
RMSE:  0.925


##################
country: Germany

horizon: 1
RMSE:  0.169

horizon: 2
RMSE:  0.677

horizon: 3
RMSE:  1.742

horizon: 6
RMSE:  3.007

horizon: 9
RMSE:  3.86

horizon: 12
RMSE:  4.793


##################
country: France

horizon: 1
RMSE:  0.056

horizon: 2
RMSE:  0.638

horizon: 3
RMSE:  1.165

horizon: 6
RMSE:  1.948

horizon: 9
RMSE:  2.31

horizon: 12
RMSE:  2.611


##################
country: Spain

horizon: 1
RMSE:  0.655

horizon: 2
RMSE:  0.771

horizon: 3
RMSE:  1.918

horizon: 6
RMSE:  2.136

horizon: 9
RMSE:  2.516

horizon: 12
RMSE:  2.196


##################
country: Italy

horizon: 1
RMSE:  0.821

horizon: 2
RMSE